In [2]:
import sys
import os
import numpy as np
from typing import Union, List, Sequence, Optional

# Add the parent directory (project root) to sys.path
script_dir              = os.path.dirname(os.curdir)
parent_dir              = os.path.abspath(os.path.join(script_dir, '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import QES.general_python.common.binary as _bin_mod
from QES.general_python.lattices.honeycomb import HoneycombLattice
from QES.general_python.lattices.square import SquareLattice
from QES.general_python.lattices.lattice import LatticeBC, Lattice
from QES.general_python.common.timer import Timer

#! General
from QES.Algebra.Operator.operator import (
    initial_states,
    test_operator_on_state,
    OperatorTypeActing,
    Operator, JAX_AVAILABLE
)

#! Spin operators
import QES.Algebra.Operator.operators_spin as op_spin
#! Fermionic operators
import QES.Algebra.Operator.operators_spinless_fermions as op_sferm


#! Backends
if JAX_AVAILABLE:
    import jax
    import jax.numpy as jnp
else:
    jax = None
    jnp = None
    
backend = 'np'
lat     = SquareLattice(dim     = 1,
                        lx      = 5,
                        ly      = 1,
                        lz      = 1,
                        bc      = LatticeBC.PBC)
test_mult = True
test_mult = False
# ------------------------------------------------------------------

12_05_2025_21-36_25 [INFO] JAX backend available and successfully imported
12_05_2025_21-36_25 [INFO] 	JAX 64-bit precision enabled.


12_05_2025_21-36_25 [INFO] Setting JAX as the active backend.
12_05_2025_21-36_25 [INFO] **************************************************
12_05_2025_21-36_25 [INFO] Backend Configuration:
12_05_2025_21-36_25 [INFO] 		NumPy Version: 1.26.4
12_05_2025_21-36_25 [INFO] 		SciPy Version: 1.13.1
12_05_2025_21-36_25 [INFO] 		JAX Version: 0.5.3
12_05_2025_21-36_25 [INFO] 		Active Backend: jax
12_05_2025_21-36_25 [INFO] 			JAX Available: True
12_05_2025_21-36_25 [INFO] 			Default Seed: 42
12_05_2025_21-36_25 [INFO] 		JAX Backend Details:
12_05_2025_21-36_25 [INFO] 				Main Module: jax.numpy
12_05_2025_21-36_25 [INFO] 				Random Module: jax.random (+ PRNGKey)
12_05_2025_21-36_25 [INFO] 				SciPy Module: jax.scipy
12_05_2025_21-36_25 [INFO] 				Default JAX Key: PRNGKey(42)
12_05_2025_21-36_25 [INFO] 		Active Data Types:
12_05_2025_21-36_25 [INFO] 				Integer Type: int64
12_05_2025_21-36_25 [INFO] 				Float Type: float64
12_05_2025_21-36_25 [INFO] 				Complex Type: complex128
12_05_2025_21-36_25 

## Spin operators $\sigma^x$, $\sigma^y$, $\sigma^z$

The Pauli matrices are defined as:
$$
\sigma^x = \begin{pmatrix}
0 & 1 \\
1 & 0
\end{pmatrix}, \quad
\sigma^y = \begin{pmatrix}
0 & -i \\
-i & 0
\end{pmatrix}, \quad
\sigma^z = \begin{pmatrix}
1 & 0 \\
0 & -1
\end{pmatrix}
$$

### a) Local operators

In [25]:
type_act = OperatorTypeActing.Local
print(f'Acting on {type_act} sites')
print(f"{type_act.value}")

sig_x_l = op_spin.sig_x(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_y_l = op_spin.sig_y(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_z_l = op_spin.sig_z(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_p_l = op_spin.sig_p(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_m_l = op_spin.sig_m(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_pm_l = op_spin.sig_pm(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_mp_l = op_spin.sig_mp(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_k_l = op_spin.sig_k(
    lattice  = lat,
    type_act = type_act,
    k        = np.pi * 2.0 / lat.ns,
    spin     = False
)
sig_x_sig_x_l = sig_x_l * sig_x_l
sig_z_sig_x_l = sig_z_l * sig_x_l
sig_x_sig_z_l = sig_x_l * sig_z_l

operators_l  = [sig_x_l, sig_y_l, sig_z_l, sig_p_l, sig_m_l, sig_pm_l, sig_mp_l, sig_k_l] if not test_mult else [sig_x_sig_x_l, sig_z_sig_x_l, sig_x_sig_z_l]
labels_l     = ['\\sigma^x', '\\sigma^y', '\\sigma^z', 
                '\\sigma^+', '\\sigma^-', 
                '[\\sigma^+]\\sigma^{\\pm}', '[\\sigma^-]\\sigma^{\\mp}', 
                '\\sigma^k'][:len(operators_l)] if not test_mult else ['(\\sigma^x\\sigma^x)', '(\\sigma^z\\sigma^x)', '(\\sigma^x\\sigma^z)']
# create a set of states and test it
int_state, np_state, jnp_state = initial_states(lat.ns, display=True)

Acting on OperatorTypeActing.Local sites
2


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### i) Integer states

In [26]:
test_operator_on_state(
    op          = operators_l,
    lat         = lat,
    state       = int_state,
    op_acting   = type_act,
    op_label    = labels_l,
    to_bin      = None,
    just_time   = True
)


Operator action: 1384526.536000 us


#### ii) Numpy states

In [6]:
test_operator_on_state(
    op          = operators_l,
    lat         = lat,
    state       = np_state,
    op_acting   = OperatorTypeActing.Local,
    op_label    = labels_l,
    to_bin      = None,
    just_time   = True
)


Operator action: 275.384000 us


#### iii) JAX states

In [39]:
test_operator_on_state(
    op          = operators_l,
    lat         = lat,
    state       = jnp_state,
    op_acting   = type_act,
    op_label    = labels_l,
    to_bin      = None,
    just_time   = False
)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Operator action: 41748.521000 us


### b) Correlation functions

In [31]:
type_act = op_spin.OperatorTypeActing.Correlation

sig_x_c = op_spin.sig_x(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_y_c = op_spin.sig_y(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_z_c = op_spin.sig_z(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_p_c = op_spin.sig_p(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_m_c = op_spin.sig_m(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_pm_c = op_spin.sig_pm(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_mp_c = op_spin.sig_mp(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
operators_c  = [sig_x_c, sig_y_c, sig_z_c, sig_p_c, sig_m_c, sig_pm_c, sig_mp_c]
labels_c     = ['\\sigma^x', '\\sigma^y', '\\sigma^z', 
                '\\sigma^+', '\\sigma^-', 
                '[\\sigma^+]\\sigma^{\\pm}', '[\\sigma^-]\\sigma^{\\mp}'][:len(operators_c)]
# create a set of states and test it
int_state, np_state, jnp_state = initial_states(lat.ns, display=True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### i) Integer states

In [14]:
test_operator_on_state(
    op          = operators_c,
    lat         = lat,
    state       = int_state,
    op_acting   = OperatorTypeActing.Correlation,
    op_label    = labels_c,
    to_bin      = None,
    just_time   = False
)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Operator action: 2147714.042000 us


### c) Global functions

In [24]:
type_act = op_spin.OperatorTypeActing.Global
sites    = [0, 3]
sig_x_g = op_spin.sig_x(
    lattice  = lat,
    type_act = type_act,
    spin     = False,
    sites    = sites
)
sig_y_g = op_spin.sig_y(
    lattice  = lat,
    type_act = type_act,
    spin     = False,
    sites    = sites
)
sig_z_g = op_spin.sig_z(
    lattice  = lat,
    type_act = type_act,
    spin     = False,
    sites    = sites
)
sig_p_g = op_spin.sig_p(
    lattice  = lat,
    type_act = type_act,
    spin     = False,
    sites    = sites
)
sig_m_g = op_spin.sig_m(
    lattice  = lat,
    type_act = type_act,
    spin     = False,
    sites    = sites
)
sig_pm_g = op_spin.sig_pm(
    lattice  = lat,
    type_act = type_act,
    spin     = False,
    sites    = sites
)
sig_mp_g = op_spin.sig_mp(
    lattice  = lat,
    type_act = type_act,
    spin     = False,
    sites    = sites
)
sig_k_g = op_spin.sig_k(
    lattice  = lat,
    type_act = type_act,
    k        = np.pi * 2.0 / lat.ns,
    spin     = False,
    sites    = sites
)
sig_x_sig_x_g = sig_x_g * sig_x_g
sig_z_sig_x_g = sig_z_g * sig_x_g
sig_x_sig_z_g = sig_x_g * sig_z_g

operators_g = [sig_x_g, sig_y_g, sig_z_g, sig_p_g, sig_m_g, sig_pm_g, sig_mp_g, sig_k_g] if not test_mult else [sig_x_sig_x_g, sig_z_sig_x_g, sig_x_sig_z_g]
labels_g     = ['\\sigma^x', '\\sigma^y', '\\sigma^z', 
                '\\sigma^+', '\\sigma^-', 
                '[\\sigma^+]\\sigma^{\\pm}', '[\\sigma^-]\\sigma^{\\mp}',
                '\\sigma^k'][:len(operators_g)] if not test_mult else ['(\\sigma^x\\sigma^x)', '(\\sigma^z\\sigma^x)', '(\\sigma^x\\sigma^z)']
# create a set of states and test it
int_state, np_state, jnp_state = initial_states(lat.ns, display=True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### i) Integer states

In [34]:
test_operator_on_state(
    op          = operators_g,
    lat         = lat,
    state       = int_state,
    op_acting   = type_act,
    op_label    = labels_g,
    to_bin      = None,
    just_time   = True,
    sites       = sites
)

Operator action: 125.703000 us


#### ii) Numpy states

In [36]:
test_operator_on_state(
    op          = operators_g,
    lat         = lat,
    state       = np_state,
    op_acting   = type_act,
    op_label    = labels_g,
    to_bin      = None,
    just_time   = True
)

Operator action: 124.422000 us


#### iii) JAX states

In [23]:
test_operator_on_state(
    op          = operators_g,
    lat         = lat,
    state       = jnp_state,
    op_acting   = type_act,
    op_label    = labels_g,
    to_bin      = None,
    just_time   = True,
    sites       = sites
)

Operator action: 747.891000 us


## Creation and annihilation operators (spinless fermions)
The creation and annihilation operators for spinless fermions are defined as:
$$
c_i^\dagger = \begin{pmatrix}
1 & 0 \\
0 & 0
\end{pmatrix}, \quad
c_i = \begin{pmatrix}
0 & 0 \\
0 & 1
\end{pmatrix}
$$
where $c_i^\dagger$ creates a fermion at site $i$ and $c_i$ annihilates a fermion at site $i$.

### a) Local operators

In [45]:
type_act = op_sferm.OperatorTypeActing.Local

c_l = op_sferm.c(
    lattice  = lat,
    ns       = lat.ns,
    type_act = type_act,
)
c_dag_l = op_sferm.cdag(
    lattice  = lat,
    ns       = lat.ns,
    type_act = type_act,
)
c_c_dag_l   = c_l * c_dag_l
n_l = op_sferm.n(
    lattice  = lat,
    ns       = lat.ns,
    type_act = type_act,
)
operators_l  = [c_l, c_dag_l, n_l]#, c_c_dag_l]
labels_l     = ['c', 'c^\\dag', 'n', '(c * c^\\dag)'][:len(operators_l)]

# create a set of states and test it
int_state, np_state, jnp_state = initial_states(lat.ns, display=True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### i) Integer states

In [47]:
test_operator_on_state(
    op          = operators_l,
    lat         = lat,
    state       = int_state,
    op_acting   = op_sferm.OperatorTypeActing.Local,
    op_label    = labels_l,
    to_bin      = None,
    # just_time   = True
)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Operator action: 9760.315000 us


#### ii) Numpy states

In [50]:
test_operator_on_state(
    op          = operators_l,
    lat         = lat,
    state       = np_state,
    op_acting   = op_sferm.OperatorTypeActing.Local,
    op_label    = labels_l,
    to_bin      = None,
    just_time   = True
)

Operator action: 137.402000 us


#### iii) JAX states

In [58]:
test_operator_on_state(
    op          = operators_l,
    lat         = lat,
    state       = jnp_state,
    op_acting   = op_sferm.OperatorTypeActing.Local,
    op_label    = labels_l,
    to_bin      = None,
    just_time   = True
)

Operator action: 5395.000000 us
